In [120]:
import pandas as pd
import numpy as np

In [121]:
data = pd.read_csv("jester-data-1.csv")

In [122]:
data.drop('74', axis=1, inplace=True)
data.head()

,-7.82,8.79,-9.66,-8.16,-7.52,-8.5,-9.85,4.17,-8.98,-4.76,...,2.82.2,99.18,99.19,99.20,99.21,99.22,-5.63,99.23,99.24,99.25
0,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
1,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
2,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
3,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
4,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,-6.65,-1.80,...,-3.54,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45


In [123]:
total_data_items = data.shape[0] * data.shape[1]
print(data.shape[0])
print(data.shape[1])
print(total_data_items)
print(data.iloc[0, 1])

24982
100
2498200
-0.29


In [125]:
count = 0
validation_data = {}
while count < (total_data_items / 10):
    row = np.random.randint(0, 24982)
    column = np.random.randint(0, 100)
    if data.iloc[row, column] != 99:
        validation_data.update({(row, column):data.iloc[row, column]})
        data.iloc[row, column] = 99
        count+=1

In [126]:
data.head()

,-7.82,8.79,-9.66,-8.16,-7.52,-8.5,-9.85,4.17,-8.98,-4.76,...,2.82.2,99.18,99.19,99.20,99.21,99.22,-5.63,99.23,99.24,99.25
0,4.08,-0.29,6.36,99.00,-2.38,-9.66,-0.73,-5.34,8.88,9.22,...,2.82,99.00,-0.29,7.86,-0.19,99.00,3.06,0.34,-4.32,1.07
1,99.00,99.00,99.00,99.00,99.00,9.27,99.00,9.27,99.00,99.00,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
2,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00
3,8.50,99.00,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,99.00,1.60
4,-6.17,-3.54,0.44,99.00,-7.09,-4.32,-8.69,99.00,-6.65,-1.80,...,-3.54,-6.89,99.00,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45


In [127]:
print(list(validation_data.keys())[0:10])

[(21498, 64), (17795, 47), (20493, 25), (1803, 37), (10930, 59), (14944, 49), (3611, 91), (17541, 31), (9862, 56), (9222, 0)]


In [131]:
n_features = 1000

user_ratings = data.values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))

In [132]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0002):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    
def validation():
    error = []
    for user_id in range(0,latent_user_preferences.shape[0]):
        for item_id in range(0,latent_item_features.shape[0]):
            rating = user_ratings[user_id][item_id]
            if (rating == 99) and ((user_id, item_id) in validation_data):
                prediction_rating = predict_rating(user_id, item_id)
                error.append(abs(validation_data[(user_id, item_id)] - prediction_rating))
    mse = (np.array(error) ** 2).mean()
    print("Validation MSE: " + str(mse))

def sgd(iterations = 10):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        print("Iteration: " + str(iteration))
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if((not np.isnan(rating)) and (rating != 99)):
                    err = train(user_id,item_id,rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        print("Train MSE: " + str(mse))
        validation()
                    


In [133]:
sgd()

Iteration: 0
Train MSE: 71.97314406093214
Validation MSE: 29.843520187099603
Iteration: 1
Train MSE: 26.953054597407046
Validation MSE: 27.574981643829553
Iteration: 2
Train MSE: 24.101413282323232
Validation MSE: 25.68605568938286
Iteration: 3
Train MSE: 21.714755593938076
Validation MSE: 23.980378479447836
Iteration: 4
Train MSE: 19.7736392443214
Validation MSE: 22.73620661104313
Iteration: 5
Train MSE: 18.432060552422143
Validation MSE: 22.014354277938583
Iteration: 6
Train MSE: 17.572221313151534
Validation MSE: 21.625937017751344
Iteration: 7
Train MSE: 16.980548964977253
Validation MSE: 21.40224731432377
Iteration: 8
Train MSE: 16.515831955093322
Validation MSE: 21.255242703154273
Iteration: 9
Train MSE: 16.107155364315002
Validation MSE: 21.145450034828855
